In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [3]:
train_df = pd.read_csv('../../final/data/train_set_noisy_4_missing_2.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.786358,21.598977,2.609478,-1.000000,-1.000000,75.211769,242.694150,4.701269,0,1.491966,135.606446,123.227658,59.636941,0.56255,69.236844,35.359073,-1.00000,4.0
1,9.765445,95.948185,-1.000000,-1.000000,473.810060,79.529629,-1.000000,3.683701,1,0.756766,86.763989,-1.000000,-1.000000,-1.00000,-1.000000,29.296335,-1.00000,4.0
2,10.581471,264.299544,4.479510,-1.000000,232.378777,86.962200,207.714187,3.650369,1,0.887986,30.558329,-1.000000,-1.000000,-1.00000,122.595648,31.744414,89.38604,5.0
3,8.455727,339.601244,-1.000000,4.201679,-1.000000,-1.000000,-1.000000,3.240530,0,-1.000000,29.899966,-1.000000,-1.000000,-1.00000,-1.000000,25.367180,-1.00000,3.0
4,7.347738,61.963008,-1.000000,-1.000000,472.934836,78.060937,-1.000000,-1.000000,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,-1.00000,4.0


In [4]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [5]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [6]:
train_df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
gender                   0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
hematocrit               0
tsat                     0
label                    0
dtype: int64

In [7]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [ ]:
for steps in [int(12e6), int(13e6), int(14e6)]:
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../../final/models/dqn_noisy_4_missing_2_{steps}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 2.85     |
|    ep_rew_mean      | -0.82    |
|    exploration_rate | 0.774    |
|    success_rate     | 0.12     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 1500     |
|    time_elapsed     | 190      |
|    total_timesteps  | 285152   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 3.37     |
|    n_updates        | 58787    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.72     |
|    ep_rew_mean      | -0.9     |
|    exploration_rate | 0.513    |
|    success_rate     | 0.09     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 1345     |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.52     |
|    ep_rew_mean      | -0.44    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.28     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 978      |
|    time_elapsed     | 5507     |
|    total_timesteps  | 5389708  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.169    |
|    n_updates        | 1334926  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.12     |
|    ep_rew_mean      | -0.4     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.34     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 963      |
|    time_elapsed     | 5953     |
|    total_timesteps  | 5738621  |
| train/              |          |
|    learning_rate  

In [ ]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_4_missing_1_7500000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
#utils.diagnose_sample(dqn_model, X_test, y_test, 1)

In [ ]:
# test_df[(test_df.y_pred==1) & (test_df.y_actual==1)]

In [ ]:
test_df.y_pred.value_counts()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df.y_pred.unique()

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/test_df3_noisy_1_11000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/success_df3_noisy_1_11000000.csv', index=False)